In [2]:
%pip install tflearn -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\igadi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [4]:
with open('intents.json') as f:
    data = json.load(f)

In [5]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


  if intent["tag"] not in labels:
    labels.append(intent["tag"])  

words = [stemmer.stem(w.lower()) for w in words if  w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
model.fit(training, output, n_epoch = 1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 23999  | total loss: 0.00001 | time: 0.091s
| Adam | epoch: 1000 | loss: 0.00001 - acc: 1.0000 -- iter: 184/188
Training Step: 24000  | total loss: 0.00001 | time: 0.094s
| Adam | epoch: 1000 | loss: 0.00001 - acc: 1.0000 -- iter: 188/188
--
INFO:tensorflow:c:\Users\igadi\OneDrive\Documents\GitHub\seabirds\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [28]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

def chat():
  print("\033[ Hello Dr. Atul! Welcome to the Medical ChatBot. Type 'quit' to exit")
  while True:
    inp = input("\033[You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      
      if tg['tag'] == tag:
        
        responses = tg['responses']

        response_list = nltk.sent_tokenize(str(responses[0]))
        colors = [31, 32, 33, 34, 35]

        for i in range(len(response_list)):
            color_index = i % len(colors)
            print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')

In [ ]:
chat ()